In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import glob
import os

from vmodel.util import color as vcolor

In [ ]:

plt.rcParams['figure.figsize'] = [4, 3.5]
plt.rcParams['figure.autolayout'] = True

In [ ]:
data_dir = '/home/fabian/vmodel_datasets/range_noise'

In [ ]:
paths = sorted(glob.glob(data_dir + '/**/merged_*.nc', recursive=True))
paths

In [ ]:
# Read datasets
expdict = {}
for path in paths:
    expname = os.path.split(os.path.dirname(path))[-1]
    ds = xr.open_dataset(path)
    expdict[expname] = ds

In [ ]:
# Concatenate datasets along experiment name
ds = xr.concat(expdict.values(), 'exp')
ds.coords['exp'] = list(expdict.keys())

In [ ]:
# Precompute distance metrics
ds['nndist_min'] = ds.nndist.min('agent')
ds['nndist_mean'] = ds.nndist.mean('agent')
ds['nndist_std'] = ds.nndist.std('agent')

In [ ]:
ds['nvisible_mean'] = ds.nvisible.mean('agent')
ds['nvisible_std'] = ds.nvisible.std('agent')

In [ ]:
labels = {
    'metric': 'metric',
    'visual': 'visual',
    'visual_myopic': 'visual + myopic',
    'visual_topo6': 'visual + topological',
    'visual_voronoi': 'visual + voronoi',
}

In [ ]:
colors = {
    'metric': vcolor.metric,
    'visual': vcolor.visual,
    'visual_myopic': vcolor.myopic,
    'visual_topo6': vcolor.topological,
    'visual_voronoi': vcolor.voronoi,
}

In [ ]:
expname = 'range_std'

In [ ]:
timeslice = slice(150, 200)

In [ ]:
def plot(ax, da, verbose=False):
    for exp in da.exp.data:
        data = da.sel(exp=exp)
        xs, ys, yerrs = data.rngstd, data.mean('run'), data.std('run')
        label, color = labels[exp], colors[exp]
        ax.errorbar(xs, ys, yerrs, fmt='-o', capsize=3, label=label, color=color)
        if verbose:
            print(f'{exp}')
            print(f'xs: {xs.data}')
            print(f'ys: {np.round(ys.data, 2)}')
            print(f'ye: {np.round(yerrs.data, 2)}')
            print()
    # ax.set(xscale='log')
    ax.set(xticks=ds.rngstd, xticklabels=ds.rngstd.data)
    ax.set(xlabel=r'range noise $\sigma_d$ [$m$]')
    ax.grid()
    ax.legend()

## Distance

In [ ]:
fig, ax = plt.subplots()
plot(ax, ds.nndist_min.isel(time=timeslice).min('time'))
ax.set(ylabel=r'min. nearest neighbor distance $d^\mathrm{min}$ [$m$]')
ax.set(ylim=(-0.15, None))
# ax.plot(ds.nagents.data, np.full(len(ds.nagents), 0.5), label='collision distance', ls='--', color='tab:red')
ax.legend()
fig.savefig(f'{expname}_nndist_min.pdf')
pass

## Order

In [ ]:
fig, ax = plt.subplots()
plot(ax, ds.order.isel(time=timeslice).mean('time'))
ax.set(ylabel=r'avg. order $\phi^{\mathrm{order}}$')
fig.savefig(f'{expname}_order.pdf')
ax.set(ylim=(-0.15, None))
pass

## Union

In [ ]:
fig, ax = plt.subplots()
plot(ax, ds.union.isel(time=timeslice).mean('time'))
ax.set(ylabel=r'avg. union $\phi^{union}$')
ax.set(ylim=(0.9, None))
fig.savefig(f'{expname}_union.pdf')
pass

## Travel

In [ ]:
# ds.traveldist.mean('run').plot(hue='exp')
# ds.traveldist.mean('run').plot(hue='exp')
fig, ax = plt.subplots()
plot(ax, ds.traveldist)
ax.set(ylabel=r'avg. travel distance $d^\mathrm{travel}$ [$m$]')
fig.savefig(f'{expname}_traveldist.pdf')
pass